## Logistic Regression

Logistic Regression is very similar to linear regression, except all of the points can only have $y$-values of $1$ or $0$. This is useful if we want to predict whether something is or isn't part of a particular class. Instead of fitting a line (as in linear regression), logistic regression involves fitting a probability curve.

For example, using our device traffic, let's see whether we can predict a DNS packet is request or response from its length. 

First, let's import the data, extract only the DNS packets, and view the first few packets.

In [ ]:
pcap = pcap_to_pandas('example_pcaps/google_home.pcap') # this can take a few minutes

# Pandas Dataframe with only DNS packets
dns_packets = pcap.loc[pcap['is_dns'] == True];
dns_packets.head(n=5)

Each row in the printed data is a packet and each column is a feature of the packet.

Next let's divide the DNS packets into requests and repsonses, and convert them into points where the $x$-value is the length of the packet and $y$-value is $1$ for requests and $0$ for responses